# Adapted Wasserstein Distance between Gaussian Processes in Discrete Time

This notebook demonstrates how to compute the Adapted Wasserstein squared distance between two Gaussian processes. The computed distance serves as a benchmark for approximations in future studies.

The adapted distance is based on:
- "Adapted optimal transport between Gaussian processes in discrete time" by Madhu Gunasingam and Ting-Kam Leonard Wong (2025).  
Reference: https://arxiv.org/abs/2404.06625

- For the multidimensional generalization, see "Entropic adapted Wasserstein distance on Gaussians" by Beatrice Acciaio, Songyan Hou, Gudmund Pammer.  
Reference: https://arxiv.org/abs/2412.18794


In [1]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

notebooks_path = os.path.abspath(os.getcwd()) 
src_path = os.path.abspath(os.path.join(notebooks_path, "../src"))

if src_path not in sys.path:
    sys.path.insert(0, src_path)

from benchmark_value_gaussian.Comp_AWD2_Gaussian import *

## Discrete-Time Brownian Motion for $d = 1$

In the one-dimensional case, we assume a process where:
- $x_0 = a$ and $x_{t+1} = x_t + \gamma_t$ with $\gamma_t \sim \mathcal{N}(0, \text{var}_a)$,
- $y_0 = b$ and $y_{t+1} = y_t + \delta_t$ with $\delta_t \sim \mathcal{N}(0, \text{var}_b)$.


In [2]:
# Parameters
a, b = 1, 2
var_a, var_b = 1**2, 0.5**2
t = 5

# Build mean and covariance matrices for both processes
a_vec, A_mat = build_mean_and_cov(t, mean_val=a, var_factor=var_a)
b_vec, B_mat = build_mean_and_cov(t, mean_val=b, var_factor=var_b)

print("Mean vector for process 1:", a_vec)
print("Covariance matrix for process 2:\n", B_mat)

# Compute adapted Wasserstein squared distance (one-dimensional)
distance_squared = adapted_wasserstein_squared_1d(A_mat, B_mat, a_vec, b_vec)
distance = np.sqrt(distance_squared)

print(f"Adapted Wasserstein Squared Distance (d=1): {distance_squared:.4f}")
print(f"Adapted Wasserstein Distance (d=1): {distance:.4f}")

Mean vector for process 1: [1 1 1 1 1]
Covariance matrix for process 2:
 [[0.25 0.25 0.25 0.25 0.25]
 [0.25 0.5  0.5  0.5  0.5 ]
 [0.25 0.5  0.75 0.75 0.75]
 [0.25 0.5  0.75 1.   1.  ]
 [0.25 0.5  0.75 1.   1.25]]
Adapted Wasserstein Squared Distance (d=1): 8.7500
Adapted Wasserstein Distance (d=1): 2.9580


## Discrete-Time Brownian Motion for $d > 1$

For the multidimensional case, we consider a process in $\mathbb{R}^{dT}$. The full covariance matrices are built using a block structure, where each block $(i, j)$ is defined as:

$$
\text{Block}(i, j) = \min(i+1, j+1) \times (\text{var} \times I_d)
$$

In [4]:
# Parameters
d = 3
T = 3

dim = d * T
a = np.ones(dim) * 1
b = np.ones(dim) * 2

A = build_full_covariance(1.1**2, d, T)
B = build_full_covariance(0.1**2, d, T)

distance_aw2 = adapted_wasserstein_squared_multidim(A, B, d, T, a, b)

print("Theoretical Adapted Wasserstein Squared Distance (d > 1):", distance_aw2)

Theoretical Adapted Wasserstein Squared Distance (d > 1): 27.0


# Non–Brownian Gaussian Process Example

For this example, we define two factor matrices $L_0$ and $M_0$ (both of size $6 \times 6$ corresponding to $d=2$ and $T=3$, so that the total dimension is $dT=6$). From these matrices, we compute:
  
- $A_0 = L_0 L_0^T$ and $B_0 = M_0 M_0^T$
- The covariance matrices are then given by $A = L L^T$ and $B = M M^T$.

We assume zero mean for both processes.

In [5]:
normalize = False

L0 = np.array([
    [1, 0, 0, 0, 0, 0],
    [1, 2, 0, 0, 0, 0],
    [1, 2, 3, 0, 0, 0],
    [7, 5, 4, 9, 0, 0],
    [6, 2, 9, 14, 16, 0],
    [-4, 6, -5, 1, 2, 9]
])
A0 = L0 @ L0.T
L = L0 / np.sqrt(np.trace(A0)) if normalize else L0
A = L @ L.T

M0 = np.array([
    [2, 0, 0, 0, 0, 0],
    [3, 1, 0, 0, 0, 0],
    [1, 4, 2, 0, 0, 0],
    [8, 5, 3, 7, 0, 0],
    [4, 3, 6, 10, 12, 0],
    [0, -1, 2, 4, 3, 8]
])
B0 = M0 @ M0.T
M = M0 / np.sqrt(np.trace(B0)) if normalize else M0
B = M @ M.T

d = 2
T = 3

dim = d * T
a = np.zeros(dim)
b = np.zeros(dim)

distance_aw2 = adapted_wasserstein_squared_multidim(A, B, d, T)

print("Adapted Wasserstein Squared Distance for custom Gaussian process:", distance_aw2)

Adapted Wasserstein Squared Distance for custom Gaussian process: 171.61705298017796
